In [ ]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop_words = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    return " ".join(tokens)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Lista de documentos preprocesados
documents = ["este es el primer documento", "este es el segundo documento", "y este es el tercero documento", "¿es este el primer documento?"]

# Crear un objeto TfidfVectorizer
vectorizer = TfidfVectorizer()

# Crear la matriz TF-IDF
tf_idf_matrix = vectorizer.fit_transform(documents)

# Obtener el vocabulario
vocab = vectorizer.get_feature_names_out()

# Imprimir la matriz TF-IDF
print(tf_idf_matrix.toarray())

[[0.39896105 0.39896105 0.39896105 0.39896105 0.60276058 0.
  0.        ]
 [0.361028   0.361028   0.361028   0.361028   0.         0.69183461
  0.        ]
 [0.361028   0.361028   0.361028   0.361028   0.         0.
  0.69183461]
 [0.39896105 0.39896105 0.39896105 0.39896105 0.60276058 0.
  0.        ]]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Lista de preguntas etiquetadas
preguntas = ["¿Cómo se hace un pastel?", "¿Qué es el cambio climático?", "¿Cómo puedo aprender programación?", "¿Cuál es la capital de México?", "¿Qué es la energía solar?", "¿Cuál es el mejor restaurante en Madrid?"]
etiquetas = ["Cocina", "Ciencia", "Programación", "Geografía", "Ciencia", "Gastronomía"]

# Dividir las preguntas en conjuntos de entrenamiento y prueba
train_preguntas, test_preguntas, train_etiquetas, test_etiquetas = train_test_split(preguntas, etiquetas, test_size=0.2, random_state=42)

# Crear un objeto TfidfVectorizer
vectorizer = TfidfVectorizer()

# Crear la matriz TF-IDF para el conjunto de entrenamiento
train_tf_idf_matrix = vectorizer.fit_transform(train_preguntas)

# Entrenar un modelo LinearSVC
model = LinearSVC()
model.fit(train_tf_idf_matrix, train_etiquetas)

# Crear la matriz TF-IDF para el conjunto de prueba
test_tf_idf_matrix = vectorizer.transform(test_preguntas)

# Evaluar el modelo
accuracy = model.score(test_tf_idf_matrix, test_etiquetas)
print(f"La precisión del modelo es: {accuracy}")

La precisión del modelo es: 0.5


In [ ]:
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Definir preguntas y etiquetas
preguntas = ["¿Cómo estás?", "¿Qué hora es?", "¿Cómo te llamas?", "¿Cuál es la capital de Francia?"]
etiquetas = ["Saludos", "Tiempo", "Nombre", "Geografía"]

# Preprocesamiento
def preprocesar(texto):
    # Pasar todo a minúsculas
    texto = texto.lower()
    # Quitar puntuación
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    # Quitar números
    texto = re.sub(r'\d+', '', texto)
    # Quitar espacios en blanco adicionales
    texto = " ".join(texto.split())
    return texto

preguntas_procesadas = [preprocesar(pregunta) for pregunta in preguntas]

# Crear matriz TF-IDF
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform(preguntas_procesadas)

# Entrenar clasificador SVM
clf = LinearSVC()
clf.fit(matriz_tfidf, etiquetas)

# Hacer predicciones
preguntas_nuevas = ["¿Cómo se llama tu perro?", "¿Qué día es hoy?", "¿Cuál es tu comida favorita?"]
preguntas_nuevas_procesadas = [preprocesar(pregunta) for pregunta in preguntas_nuevas]
matriz_tfidf_nuevas = vectorizer.transform(preguntas_nuevas_procesadas)
predicciones = clf.predict(matriz_tfidf_nuevas)

# Imprimir resultados
for pregunta, etiqueta in zip(preguntas_nuevas, predicciones):
    print(f"Pregunta: {pregunta}\nEtiqueta: {etiqueta}\n") 

Pregunta: ¿Cómo se llama tu perro?
Etiqueta: Saludos

Pregunta: ¿Qué día es hoy?
Etiqueta: Tiempo

Pregunta: ¿Cuál es tu comida favorita?
Etiqueta: Geografía



In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline


preguntas = [
    '¿Cuál es el nombre del proyecto?',
    '¿En qué lenguaje de programación está escrito el proyecto?',
    '¿Quiénes son los desarrolladores del proyecto?',
    '¿Cuál es el objetivo del proyecto?',
    '¿Cuál es la licencia del proyecto?',
    '¿Dónde puedo encontrar más información sobre el proyecto?'
]

respuestas = [
    'El nombre del proyecto es Chatbot en español.',
    'El proyecto está escrito en Python.',
    'El proyecto fue desarrollado por un grupo de estudiantes de programación.',
    'El objetivo del proyecto es crear un chatbot que pueda responder preguntas frecuentes.',
    'El proyecto está bajo la licencia MIT.',
    'Puedes encontrar más información sobre el proyecto en el archivo README.md.'
]

# Entrenamiento del modelo
vectorizer = TfidfVectorizer()
clf = SVC(C=1)

train_preguntas_preprocesadas = [re.sub(r'[^\w\s]', '', pregunta.lower()) for pregunta in preguntas]
train_etiquetas = respuestas

train_preguntas_vectorizadas = vectorizer.fit_transform(train_preguntas_preprocesadas)

clf.fit(train_preguntas_vectorizadas, train_etiquetas)

def responder(pregunta):
    pregunta_preprocesada = re.sub(r'[^\w\s]', '', pregunta.lower())
    pregunta_vectorizada = vectorizer.transform([pregunta_preprocesada])
    etiqueta_predicha = clf.predict(pregunta_vectorizada)[0]
    return etiqueta_predicha

# Ejecución del chatbot
while True:
    pregunta = input('Ingresa tu pregunta: ')
    respuesta = responder(pregunta)
    print(respuesta)